In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Multi Query

### Index


In [2]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, chunk_overlap=50
)

# Make splits
splits = text_splitter.split_documents(blog_docs)

# Index
vector_store = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(model="text-embedding-3-small"),
)

retriever = vector_store.as_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.


### Prompt


In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)


generate_queries = (
    prompt_perspectives
    | ChatOpenAI(model="gpt-4o-mini", temperature=0)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

In [4]:
from langchain.load import dumps, loads


def get_unique_union(documents: list[list]):
    """Unique union of retrieved docs"""
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]


# Retrieve
question = "What is task decomposition for LLM agents?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question": question})
len(docs)

/Users/clement/Developer/ai/langchain-notebooks/.venv/lib/python3.11/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


9

In [5]:
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain, "question": itemgetter("question")}
    | prompt
    | ChatOpenAI(model="gpt-4o-mini", temperature=0)
    | StrOutputParser()
)

final_rag_chain.invoke({"question": question})

'Task decomposition for LLM (large language model) agents involves breaking down complex tasks into smaller, manageable subgoals. This process enables the agent to handle complicated tasks more efficiently. There are several methods for task decomposition:\n\n1. **Chain of Thought (CoT)**: This prompting technique encourages the model to "think step by step," allowing it to utilize more computation at test time to decompose difficult tasks into simpler steps.\n\n2. **Tree of Thoughts**: This approach extends CoT by exploring multiple reasoning possibilities at each step. It decomposes the problem into multiple thought steps and generates various thoughts per step, creating a tree structure that can be searched using breadth-first or depth-first search methods.\n\n3. **Prompting Techniques**: Task decomposition can be achieved through simple prompts, such as asking the model for the steps to achieve a goal or subgoals for a specific task.\n\n4. **Task-Specific Instructions**: Providing 

# RAG-Fusion


In [6]:
from langchain.prompts import ChatPromptTemplate

# RAG-Fusion: Related
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

generate_queries = (
    prompt_rag_fusion | model | StrOutputParser() | (lambda x: x.split("\n"))
)

In [8]:
from langchain.load import dumps, loads


def reciprocal_rank_fusion(results: list[list], k=60):
    """Reciprocal_rank_fusion that takes multiple lists of ranked documents
    and an optional parameter k used in the RRF formula"""

    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results


retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": question})
len(docs)

7

In [9]:
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion, "question": itemgetter("question")}
    | prompt
    | model
    | StrOutputParser()
)

final_rag_chain.invoke({"question": question})

'Task decomposition for LLM (large language model) agents involves breaking down complex tasks into smaller, manageable subgoals. This process enables the agent to handle complicated tasks more efficiently. There are several methods for task decomposition:\n\n1. **Prompting**: The LLM can be prompted with simple instructions like "Steps for XYZ" or "What are the subgoals for achieving XYZ?" to guide the decomposition process.\n\n2. **Task-specific instructions**: The agent can use specific instructions tailored to the task, such as "Write a story outline" for writing a novel.\n\n3. **Human inputs**: Human guidance can also be utilized to assist in breaking down tasks.\n\nAdditionally, techniques like Chain of Thought (CoT) and Tree of Thoughts extend the capabilities of task decomposition. CoT encourages the model to think step by step, while Tree of Thoughts explores multiple reasoning possibilities at each step, creating a tree structure for more comprehensive planning.'

# Decomposition


In [10]:
from langchain.prompts import ChatPromptTemplate

# Decomposition
template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
Generate multiple search queries related to: {question} \n
Output (3 queries):"""
prompt_decomposition = ChatPromptTemplate.from_template(template)

In [11]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# LLM
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Chain
generate_queries_decomposition = (
    prompt_decomposition | model | StrOutputParser() | (lambda x: x.split("\n"))
)

# Run
question = "What are the main components of an LLM-powered autonomous agent system?"
questions = generate_queries_decomposition.invoke({"question": question})

In [12]:
questions

['1. What are the key components of a large language model (LLM) in an autonomous agent system?',
 '2. How do perception and sensor integration function within an LLM-powered autonomous agent?',
 '3. What role does decision-making and planning play in the architecture of an LLM-based autonomous agent system?']

In [13]:
# Prompt
template = """Here is the question you need to answer:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question: 

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [14]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser


def format_qa_pair(question, answer):
    """Format Q and A pair"""

    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    return formatted_string.strip()


q_a_pairs = ""
for q in questions:
    rag_chain = (
        {
            "context": itemgetter("question") | retriever,
            "question": itemgetter("question"),
            "q_a_pairs": itemgetter("q_a_pairs"),
        }
        | decomposition_prompt
        | model
        | StrOutputParser()
    )

    answer = rag_chain.invoke({"question": q, "q_a_pairs": q_a_pairs})
    q_a_pair = format_qa_pair(q, answer)
    q_a_pairs = q_a_pairs + "\n---\n" + q_a_pair

In [15]:
q_a_pairs.split("\n---\n")

['',
 'Question: 1. What are the key components of a large language model (LLM) in an autonomous agent system?\nAnswer: The key components of a large language model (LLM) in an autonomous agent system include:\n\n1. **Planning**: This involves the ability of the agent to break down complex tasks into smaller, manageable subgoals. Effective planning is crucial for handling complicated tasks efficiently.\n\n2. **Subgoal and Decomposition**: The agent utilizes techniques like Chain of Thought (CoT) and Tree of Thoughts to decompose tasks into simpler steps. CoT encourages the model to think step by step, while Tree of Thoughts allows for exploring multiple reasoning possibilities at each step.\n\n3. **Reflection and Refinement**: The agent can engage in self-criticism and reflection over past actions, learning from mistakes to improve future performance. This iterative process enhances the quality of the results produced by the agent.\n\n4. **Memory**: The system may incorporate memory co

In [16]:
answer

"Decision-making and planning are fundamental components in the architecture of an LLM-based autonomous agent system, serving as the backbone for how the agent interprets tasks, formulates strategies, and executes actions. Here’s a detailed breakdown of their roles:\n\n1. **Task Decomposition**: Planning allows the agent to break down complex tasks into smaller, manageable subgoals. This decomposition is essential for efficient handling of intricate tasks, enabling the agent to approach problems systematically. Techniques such as Chain of Thought (CoT) and Tree of Thoughts facilitate this process by encouraging the agent to think step-by-step and explore multiple reasoning possibilities at each stage.\n\n2. **Strategic Decision-Making**: The planning component is responsible for generating a sequence of actions that the agent should take to achieve its goals. This involves evaluating different options, predicting outcomes, and selecting the most effective course of action. The agent mu

### Answer individually


In [17]:
# Answer each sub-question individually

from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

# RAG prompt
prompt_rag = hub.pull("rlm/rag-prompt")


def retrieve_and_rag(question, prompt_rag, sub_question_generator_chain):
    """RAG on each sub-question"""

    # Use our decomposition /
    sub_questions = sub_question_generator_chain.invoke({"question": question})

    # Initialize a list to hold RAG chain results
    rag_results = []

    for sub_question in sub_questions:

        # Retrieve documents for each sub-question
        retrieved_docs = retriever.invoke(sub_question)

        # Use retrieved documents and sub-question in RAG chain
        answer = (prompt_rag | model | StrOutputParser()).invoke(
            {"context": retrieved_docs, "question": sub_question}
        )
        rag_results.append(answer)

    return rag_results, sub_questions


# Wrap the retrieval and RAG process in a RunnableLambda for integration into a chain
answers, questions = retrieve_and_rag(
    question, prompt_rag, generate_queries_decomposition
)

In [18]:
def format_qa_pairs(questions, answers):
    """Format Q and A pairs"""

    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
    return formatted_string.strip()


context = format_qa_pairs(questions, answers)

# Prompt
template = """Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = prompt | model | StrOutputParser()

final_rag_chain.invoke({"context": context, "question": question})

'An LLM-powered autonomous agent system comprises several key components that work together to enable effective functioning. These components include:\n\n1. **Planning and Subgoal Decomposition**: The system is designed to break down complex tasks into manageable subgoals, allowing for efficient task handling and organization.\n\n2. **Decision-Making**: Decision-making processes are integral, utilizing techniques such as Chain of Thought (CoT) and Tree of Thoughts to explore various possibilities and refine approaches to tasks.\n\n3. **Perception and Sensor Integration**: The agent processes observations and events from its environment through a retrieval model, assessing the relevance and importance of these inputs to inform its behavior and planning.\n\n4. **Reflection and Refinement**: A reflection mechanism enables the agent to learn from past actions, synthesizing experiences into higher-level inferences that guide future decisions and improve overall performance.\n\n5. **Natural 

# Step back


In [19]:
# Few Shot Examples
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jan Šindel was born in what country?",
        "output": "what is Jan Šindel's personal history?",
    },
]
# We now transform these to example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. 
            Your task is to step back and paraphrase a question to a more generic step-back question, 
            which is easier to answer. Here are a few examples:""",
        ),
        # Few shot examples
        few_shot_prompt,
        # New question
        ("user", "{question}"),
    ]
)

In [20]:
generate_queries_step_back = prompt | model | StrOutputParser()
question = "What is task decomposition for LLM agents?"
generate_queries_step_back.invoke({"question": question})

'what does task decomposition mean in the context of agents?'

In [21]:
# Response prompt
response_prompt_template = """
You are an expert of world knowledge. 
I am going to ask you a question. 
Your response should be comprehensive and not contradicted with the following context if they are relevant. 
Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

chain = (
    {
        # Retrieve context using the normal question
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        # Retrieve context using the step-back question
        "step_back_context": generate_queries_step_back | retriever,
        # Pass on the question
        "question": lambda x: x["question"],
    }
    | response_prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question": question})

'Task decomposition for LLM (large language model) agents refers to the process of breaking down complex tasks into smaller, more manageable subgoals. This approach enhances the efficiency and effectiveness of the agent in handling intricate tasks. \n\nThere are several techniques and methodologies for task decomposition:\n\n1. **Chain of Thought (CoT)**: This technique encourages the model to "think step by step," allowing it to utilize more computational resources at test time. By decomposing a large task into smaller, simpler steps, CoT not only facilitates task management but also provides insights into the model\'s reasoning process.\n\n2. **Tree of Thoughts**: An extension of CoT, this method explores multiple reasoning possibilities at each step. It decomposes the problem into various thought steps and generates multiple thoughts for each step, forming a tree structure. The search process can be conducted using either breadth-first search (BFS) or depth-first search (DFS), with 

# HyDE


In [22]:
from langchain.prompts import ChatPromptTemplate

# HyDE document genration
template = """Please write a scientific paper passage to answer the question
Question: {question}
Passage:"""
prompt_hyde = ChatPromptTemplate.from_template(template)

generate_docs_for_retrieval = prompt_hyde | model | StrOutputParser()

# Run
question = "What is task decomposition for LLM agents?"
generate_docs_for_retrieval.invoke({"question": question})

"**Task Decomposition for LLM Agents**\n\nTask decomposition is a critical process in the context of Large Language Model (LLM) agents, wherein complex tasks are systematically broken down into smaller, more manageable sub-tasks. This approach enhances the efficiency and effectiveness of LLMs in performing intricate operations that require multi-step reasoning or the integration of diverse knowledge domains. \n\nIn the realm of artificial intelligence, particularly with LLMs, task decomposition serves several key purposes. Firstly, it allows for the simplification of problem-solving by enabling the model to focus on one aspect of a task at a time, thereby reducing cognitive load and minimizing the risk of errors. For instance, when tasked with generating a comprehensive report, an LLM can decompose the task into distinct components such as data collection, analysis, and synthesis of findings, allowing for a more structured and coherent output.\n\nSecondly, task decomposition facilitate

In [23]:
# Retrieve
retrieval_chain = generate_docs_for_retrieval | retriever
retrieved_docs = retrieval_chain.invoke({"question": question})
retrieved_docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search

In [24]:
# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = prompt | model | StrOutputParser()

final_rag_chain.invoke({"context": retrieved_docs, "question": question})

'Task decomposition for LLM (large language model) agents involves breaking down complex tasks into smaller, manageable subgoals. This process enables the agent to handle complicated tasks more efficiently. There are several methods for task decomposition:\n\n1. **Chain of Thought (CoT)**: This technique encourages the model to "think step by step," allowing it to utilize more computation at test time to decompose hard tasks into simpler steps. CoT transforms large tasks into multiple manageable tasks and provides insight into the model\'s reasoning process.\n\n2. **Tree of Thoughts**: This method extends CoT by exploring multiple reasoning possibilities at each step. It decomposes the problem into multiple thought steps and generates various thoughts per step, creating a tree structure. The search process can be conducted using breadth-first search (BFS) or depth-first search (DFS), with each state evaluated by a classifier or through majority voting.\n\n3. **Prompting and Instruction